In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
%store -r include_kmeans
%store -r include_PCA
%store -r include_holidays
%store -r include_wknds
%store -r include_phase
%store -r include_dayofweek

csv_name = "X"

if (include_kmeans):
    csv_name = csv_name + "_kmeans"

if(include_PCA):
    csv_name = csv_name + "_pca"

if(include_phase):
    csv_name = csv_name + "_phase"
    
if(include_holidays):
    csv_name = csv_name + "_holidays"
    
if(include_wknds):
    csv_name = csv_name + "_weekends"
    
if(include_dayofweek):
    csv_name = csv_name + "_weekdays"
    
csv_name = csv_name + ".csv"

In [3]:
df_x = pd.read_csv ('../../data/input/' + csv_name)
print(df_x)

df_y = pd.read_csv ('../../data/outputs/Y.csv')
print(df_y)

        Unnamed: 0     MeterA_1     MeterA_2     MeterA_3     MeterB_1  \
0                0  2378.388636  2376.409889  2361.400016  2408.966692   
1                1  2382.797347  2380.907281  2371.481343  2409.007524   
2                2  2379.415364  2377.256861  2366.150157  2409.164873   
3                3  2376.835701  2374.778306  2360.276611  2409.201994   
4                4  2376.486046  2374.294670  2360.364542  2407.978294   
...            ...          ...          ...          ...          ...   
175675      175675  2383.949621  2383.647127  2364.816449  2390.746154   
175676      175676  2384.643960  2384.341416  2366.016259  2394.415031   
175677      175677  2388.354823  2388.071887  2373.152195  2399.218883   
175678      175678  2388.391098  2388.068581  2373.609684  2403.011375   
175679      175679  2389.275344  2388.952592  2374.863487  2405.452361   

           MeterB_2     MeterB_3     MeterC_1     MeterC_2     MeterC_3  ...  \
0       2409.306509  2418.90850

## Inverse Neural Net

In [24]:
from numpy import linalg as LA
from sklearn.metrics import mean_squared_error
def get_scaled_MSE(y_target, y_pred):
    
    y_target = pd.DataFrame(y_target)
    y_pred = pd.DataFrame(y_pred)
    
    #finding L2 norm of the y_target vectors
    L2_ratios = []
    for index, row in y_target.iterrows():
        L2_ratios.append(LA.norm(row))
        
    #dividing all y_pred by the L2 ratios:
    ratio_index = 0
    scaled_ratio_y_pred = []
    for index, row in y_pred.iterrows():
        scaled_ratio_y_pred.append(row/L2_ratios[ratio_index])
        ratio_index += 1
        
    #dividing all y_target by the L2 ratios:
    ratio_index = 0
    scaled_ratio_y_target = []
    for index, row in y_target.iterrows():
        scaled_ratio_y_target.append(row/L2_ratios[ratio_index])
        ratio_index += 1
    
    mse = mean_squared_error(scaled_ratio_y_pred, scaled_ratio_y_target)
    
    return mse

In [5]:
from sklearn.decomposition import PCA
# get the dataset
def get_dataset():
    X, y = df_x.copy().to_numpy(), df_y.copy().to_numpy()
    X = X[172000:]
    y = y[172000:]
    X_val = X[:3680]
    y_val = y[:3680]
    if(include_PCA):
        pca = PCA(n_components=10)
        X = pca.fit_transform(X)
        X_val = pca.transform(X_val)
        #inv_pca = pca.inverse_transform(X)
    if(include_kmeans):
        kmeans = KMeans(n_clusters=5)
        X = kmeans.fit_transform(X)
        X_val = k_means.transform(X_val)
    return X, y, X_val, y_val

In [16]:
# get the model
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense
def get_inn_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(48, activation = 'sigmoid'))
    model.add(Dense(n_outputs))
    a_optimizer = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.99, epsilon=1e-06, amsgrad=False,name='Adam')
    model.compile(loss='mse', optimizer=a_optimizer)

    return model

In [10]:
# load dataset
X, y, X_val, y_val = get_dataset()

# scale target data
y = y/2400
y_val = y_val/2400

In [17]:
model = get_inn_model(X.shape[1], y.shape[1])
# fit model
model.fit(X, y, verbose=1, epochs=200)

Epoch 1/200
115/115 [==============================] - 1s 2ms/step - loss: 131.7312
Epoch 2/200
115/115 [==============================] - 0s 2ms/step - loss: 73.3597
Epoch 3/200
115/115 [==============================] - 0s 2ms/step - loss: 52.3363
Epoch 4/200
115/115 [==============================] - 0s 2ms/step - loss: 45.6376
Epoch 5/200
115/115 [==============================] - 0s 2ms/step - loss: 42.6132
Epoch 6/200
115/115 [==============================] - 0s 3ms/step - loss: 40.3188
Epoch 7/200
115/115 [==============================] - 0s 3ms/step - loss: 38.8517
Epoch 8/200
115/115 [==============================] - 0s 3ms/step - loss: 37.3497
Epoch 9/200
115/115 [==============================] - 0s 3ms/step - loss: 36.7309
Epoch 10/200
115/115 [==============================] - 0s 3ms/step - loss: 35.7941
Epoch 11/200
115/115 [==============================] - 0s 2ms/step - loss: 35.0729
Epoch 12/200
115/115 [==============================] - 0s 3ms/step - loss: 34.4616


115/115 [==============================] - 0s 3ms/step - loss: 25.8904
Epoch 99/200
115/115 [==============================] - 0s 3ms/step - loss: 25.6333
Epoch 100/200
115/115 [==============================] - 0s 3ms/step - loss: 25.6917
Epoch 101/200
115/115 [==============================] - 0s 3ms/step - loss: 25.5602
Epoch 102/200
115/115 [==============================] - 0s 3ms/step - loss: 25.4971
Epoch 103/200
115/115 [==============================] - 0s 3ms/step - loss: 25.6636
Epoch 104/200
115/115 [==============================] - 0s 3ms/step - loss: 25.6589
Epoch 105/200
115/115 [==============================] - 0s 3ms/step - loss: 25.6238
Epoch 106/200
115/115 [==============================] - 0s 2ms/step - loss: 26.1214
Epoch 107/200
115/115 [==============================] - 0s 2ms/step - loss: 25.3936
Epoch 108/200
115/115 [==============================] - 0s 2ms/step - loss: 25.4660
Epoch 109/200
115/115 [==============================] - 0s 3ms/step - loss: 25.

115/115 [==============================] - 0s 3ms/step - loss: 24.5065
Epoch 195/200
115/115 [==============================] - 0s 2ms/step - loss: 24.4123
Epoch 196/200
115/115 [==============================] - 0s 2ms/step - loss: 24.5083
Epoch 197/200
115/115 [==============================] - 0s 2ms/step - loss: 24.7580
Epoch 198/200
115/115 [==============================] - 0s 3ms/step - loss: 24.6049
Epoch 199/200
115/115 [==============================] - 0s 2ms/step - loss: 24.4642
Epoch 200/200
115/115 [==============================] - 0s 2ms/step - loss: 24.7621


In [25]:
y_pred = model.predict(X_val)
print("MSE: ", get_scaled_MSE(y_val, y_pred))

MSE:  0.002422159266358133


In [26]:
pd.DataFrame(y_pred)

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,72.001183,3.081786,6.180046,-15.438756,2.150272,-18.170881,-1.104519,5.145156,1.908676,4.123828,...,-4.384049,3.975219,0.707628,1.534881,-15.051125,3.914752,-9.305395,3.232573,-2.523150,1.490746
1,71.778168,4.484689,5.336110,-5.971019,1.575113,-3.549386,3.054715,4.005070,1.535142,6.432215,...,0.316012,3.608307,5.473745,2.568105,-6.542482,1.346216,-1.473183,1.908747,-1.368683,0.612605
2,71.910164,-1.175407,5.470561,-12.692102,3.609593,-12.219858,1.206072,5.320052,2.010067,1.068078,...,-5.042125,3.636507,3.641474,2.218271,-11.134369,3.471834,-8.108334,3.095294,-0.257494,2.330392
3,71.998215,0.534643,5.968648,-15.959958,2.363053,-17.669004,-0.393151,4.197044,1.517953,3.446203,...,-2.756783,4.868239,0.507667,1.430882,-16.528162,4.050832,-9.404812,3.489025,-3.501778,1.407687
4,71.903221,-5.938277,4.358364,-13.318407,3.731369,-11.784410,1.756994,4.495703,1.675049,0.447589,...,-5.638565,3.658768,3.363816,2.056454,-11.593749,3.656385,-8.803452,3.184648,-1.145949,2.251471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,73.001534,1.959557,2.491857,12.464801,5.701162,18.938677,8.240458,7.180246,3.347053,20.742470,...,8.998400,4.157213,29.441055,12.465064,1.801249,2.780920,1.876274,2.659122,5.029632,2.578348
3676,73.001534,1.959557,2.491857,12.464801,5.701162,18.938677,8.240458,7.180246,3.347053,20.742470,...,8.998400,4.157213,29.441055,12.465064,1.801249,2.780920,1.876274,2.659122,5.029632,2.578348
3677,73.040909,8.295600,4.581291,9.761857,4.688681,7.554971,4.194810,7.197397,3.385646,19.291910,...,3.749569,2.153903,28.315899,11.972407,3.658811,3.160915,0.724410,2.109049,8.296108,3.733212
3678,72.963486,1.021453,3.189849,7.181476,4.490320,-2.458843,1.029503,6.283069,3.014393,17.437984,...,2.391192,2.967187,23.371487,10.475285,1.097782,3.605058,-2.643820,2.292472,5.774143,3.367559


In [27]:
pd.DataFrame(y_val)

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,71.666667,-5.95625,2.708259,-10.35625,4.689507,-13.15000,4.366497,4.11250,1.844641,-5.72500,...,-12.91250,5.627950,5.70000,2.684747,-16.62500,6.039802,-9.00000,3.049819,-7.80000,2.734396
1,71.667083,-16.56250,7.744736,-10.57500,4.772102,-12.31875,5.238566,4.14375,1.758021,-5.66875,...,-12.92500,4.589630,6.16875,2.098350,-16.63125,6.336773,-9.00000,3.013351,-7.08750,3.218432
2,71.667500,-16.47500,6.680433,-10.70625,4.839266,-12.06875,4.299071,5.73125,2.474232,-5.30000,...,-12.71875,6.028980,6.30625,2.944442,-16.50625,6.682544,-8.83750,3.912436,-6.46875,3.036503
3,71.667917,-17.78125,7.897202,-9.94375,4.122675,-11.38750,4.448743,3.58750,1.562538,-5.09375,...,-11.35625,5.344385,6.26875,2.692359,-16.49375,6.298058,-8.46250,3.365660,-6.03125,2.557577
4,71.668333,-16.90000,6.940507,-9.38750,3.838331,-11.01875,4.909334,3.35625,1.405671,-5.90000,...,-10.35000,4.611266,6.40000,2.486071,-15.88750,6.114208,-7.99375,3.841325,-5.58125,2.522069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,73.197917,2.69375,1.175966,19.06875,7.718403,21.04375,9.386708,3.51250,1.162197,23.93750,...,1.93750,0.753049,26.93125,11.299582,1.97500,0.760214,17.16250,5.757408,2.79375,1.099051
3676,73.198333,3.21875,1.285784,19.05625,8.345402,10.01250,4.247231,9.85000,4.450601,24.10625,...,1.93125,0.825799,26.15625,12.227977,1.96875,0.822980,16.63125,7.022371,2.76875,1.036728
3677,73.198750,3.18750,1.515161,19.08750,7.243901,7.50000,2.620333,16.13750,7.022757,24.69375,...,1.93125,0.834854,26.08125,12.133290,1.29375,0.545320,16.40625,7.740735,2.76875,1.161328
3678,73.199167,3.18125,1.242479,19.19375,7.751827,3.50625,1.195334,16.38125,7.607421,24.52500,...,1.92500,0.827160,25.61250,10.742130,1.14375,0.481027,16.38125,6.743518,2.76875,1.320728
